In [ ]:
import os
import sys
import shutil
import argparse
import tempfile
import urllib.request
import zipfile
import json

In [ ]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']
!pip install tensorflow==1.15
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 13.68 MiB/s, done.
Resolving deltas: 100% (182/182), done.
     |████████████████████████████████| 412.3 MB 19 kB/s 
     |████████████████████████████████| 503 kB 48.8 MB/s 
     |████████████████████████████████| 3.8 MB 51.8 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bc8d457ac01023ad3416b3e230765cfbd9636eaa02b2dfea350c115a834c1c46
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found 

In [ ]:
! wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip

--2021-08-14 06:55:04--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M   151MB/s    in 2.6s    

2021-08-14 06:55:07 (151 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]



In [ ]:
with zipfile.ZipFile('/content/uncased_L-12_H-768_A-12.zip', 'r') as f:
  f.extractall('/content/BERT_weights')

In [ ]:
!git clone https://github.com/kwonmha/bert-vocab-builder.git
!python /content/bert-vocab-builder/subword_builder.py \
  --corpus_filepattern "/content/train.sent" \
  --output_filename "vocab.txt"\
  --min_count 5

fatal: destination path 'bert-vocab-builder' already exists and is not an empty directory.


W0814 06:56:24.783973 139750691219328 module_wrapper.py:139] From /content/bert-vocab-builder/subword_builder.py:58: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0814 06:56:24.784270 139750691219328 module_wrapper.py:139] From /content/bert-vocab-builder/tokenizer.py:135: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

The number of files to read :  1

W0814 06:56:24.785339 139750691219328 module_wrapper.py:139] From /content/bert-vocab-builder/tokenizer.py:139: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/

In [ ]:
!python /content/bert_repo/create_pretraining_data.py \
  --input_file=/content/train.sent\
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=/content/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5



W0814 06:58:25.343713 139823777904512 module_wrapper.py:139] From /content/bert_repo/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0814 06:58:25.343905 139823777904512 module_wrapper.py:139] From /content/bert_repo/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0814 06:58:25.344081 139823777904512 module_wrapper.py:139] From /content/bert_repo/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0814 06:58:25.354501 139823777904512 module_wrapper.py:139] From /content/bert_repo/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0814 06:58:25.355633 139823777904512 module_wrapper.py:139] From /content/bert_repo/create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.in

In [ ]:
!python /content/bert_repo/run_pretraining.py \
  --input_file=/tmp/tf_examples.tfrecord \
  --output_dir=/content/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/content/BERT_weights/bert_config.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-4 \
  --num_train_steps=2 \
  --iterations_per_loop=1




W0814 07:00:50.606647 140572738783104 module_wrapper.py:139] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0814 07:00:50.606865 140572738783104 module_wrapper.py:139] From /content/bert_repo/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0814 07:00:50.607073 140572738783104 module_wrapper.py:139] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0814 07:00:50.608496 140572738783104 module_wrapper.py:139] From /content/bert_repo/run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0814 07:00:50.608703 140572738783104 module_wrapper.py:139] From /content/bert_repo/run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0814 07:00:50.609834 140

In [ ]:
!python /content/bert_repo/extract_features.py \
  --input_file=/content/train.sent\
  --output_file=/tmp/output.jsonl \
  --vocab_file=/content/vocab.txt \
  --bert_config_file=/content/BERT_weights/bert_config.json \
  --init_checkpoint=/content/pretraining_output/model.ckpt-0 \
  --layers=-1 \
  --max_seq_length=256 \
  --batch_size=8



W0814 07:01:29.980840 139645753460608 module_wrapper.py:139] From /content/bert_repo/extract_features.py:344: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0814 07:01:29.981031 139645753460608 module_wrapper.py:139] From /content/bert_repo/extract_features.py:344: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0814 07:01:29.981221 139645753460608 module_wrapper.py:139] From /content/bert_repo/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0814 07:01:29.991137 139645753460608 laz

In [ ]:
embedding = []
with open('/tmp/output.jsonl') as f:
    for line in f:
        embedding.append(json.loads(line))

In [ ]:
token_text=[]
token_embedding=[]
for sentence_index in range(3428):
  features = embedding[sentence_index]["features"]
  for feature in features:
    if (feature["token"] not in token_text):
      token_text.append(feature["token"])
      token_embedding.append(feature["layers"][0]["values"])
    else:
      continue

In [ ]:
f=open("bert2vec.txt","w")
for i in range(len(token_text)):
  f.write(token_text[i]+" ")
  for j in range(768):
    f.write(str(token_embedding[i][j])+" ")
  f.write("\n")
f.close()